In [1]:
import os
os.chdir("C:/Users/Anushka/Desktop/anushka/ML/projects_vscode/IPL prediction system")

In [3]:
import pandas as pd
import os

base_dir = os.getcwd()
matches = pd.read_csv(os.path.join(base_dir, 'data_processing\cleaned_matches.csv'))
deliveries = pd.read_csv(os.path.join(base_dir, 'data_processing\cleaned_deliveries.csv'))


In [4]:
from sklearn.preprocessing import LabelEncoder

matches['toss_winner_match_winner'] = (matches['toss_winner'] == matches['winner']).astype(int)

venue_encoder = LabelEncoder()
matches['venue_encoded'] = venue_encoder.fit_transform(matches['venue'].astype(str))

matches['is_home_team'] = (matches['team1'] == matches['city']).astype(int)


In [5]:
import pandas as pd
import os

matches = pd.read_csv('data_processing\cleaned_matches.csv')
deliveries = pd.read_csv('data_processing\cleaned_deliveries.csv')
matches.rename(columns={'id': 'match_id'}, inplace=True)
merged_df = deliveries.merge(matches, on='match_id')


merged_df = deliveries.merge(matches, on='match_id')

os.makedirs('data_processing/processed_data', exist_ok=True)
merged_df.to_csv('data_processing/processed_data/merged.csv', index=False)

merged_df.head()


,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,...,toss_decision,winner,result,result_margin,target_runs,target_overs,super_over,method,umpire1,umpire2
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC GANGULY,P KUMAR,BB McCullum,0,...,field,KOLKATA KNIGHT RIDERS,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB MCCULLUM,P KUMAR,SC Ganguly,0,...,field,KOLKATA KNIGHT RIDERS,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB MCCULLUM,P KUMAR,SC Ganguly,0,...,field,KOLKATA KNIGHT RIDERS,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB MCCULLUM,P KUMAR,SC Ganguly,0,...,field,KOLKATA KNIGHT RIDERS,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB MCCULLUM,P KUMAR,SC Ganguly,0,...,field,KOLKATA KNIGHT RIDERS,runs,140.0,223.0,20.0,N,NaN,Asad Rauf,RE Koertzen


In [6]:
second_innings_df = merged_df[merged_df['inning'] == 2].copy()


In [8]:
context_df = second_innings_df[['match_id', 'venue', 'batting_team', 'bowling_team', 'over', 'ball', 'total_runs']].copy()
context_df['current_score'] = context_df.groupby('match_id')['total_runs'].cumsum()
context_df['ball_number'] = (context_df['over'] - 1) * 6 + context_df['ball']
context_df['balls_left'] = 120 - context_df['ball_number']


In [9]:
context_df = context_df.merge(second_innings_df[['match_id', 'over', 'ball', 'player_dismissed']], 
                              on=['match_id', 'over', 'ball'], 
                              how='left')
context_df['player_dismissed'] = context_df['player_dismissed'].fillna(0).apply(lambda x: 0 if x == 0 else 1)
context_df['wickets_lost'] = context_df.groupby('match_id')['player_dismissed'].cumsum()
context_df['wickets_left'] = 10 - context_df['wickets_lost']


In [14]:
first_innings_df = merged_df[merged_df['inning'] == 1]


In [32]:
second_innings_df = merged_df[merged_df['inning'] == 2]

final_scores = second_innings_df.groupby('match_id')['total_runs'].sum().reset_index()
final_scores.rename(columns={'total_runs': 'second_innings_score'}, inplace=True)

results_df = targets.merge(final_scores, on='match_id')
results_df['result'] = results_df['second_innings_score'] >= results_df['target']
results_df['result'] = results_df['result'].map({True: 1, False: 0})  # 1 = won, 0 = lost

context_df = context_df.merge(results_df[['match_id', 'result']], on='match_id')


In [33]:
matches = pd.read_csv('data_processing/cleaned_matches.csv')
deliveries = pd.read_csv('data_processing/cleaned_deliveries.csv')

# Merge using 'id' from matches and 'match_id' from deliveries
merged_df = deliveries.merge(matches, left_on='match_id', right_on='id')

total_runs = merged_df.groupby(['match_id', 'batting_team'])['total_runs'].sum().reset_index()
total_runs.rename(columns={'total_runs': 'innings_total_runs'}, inplace=True)

# Feature 2: Calculate number of wickets lost per team per match
wicket_kinds = ['caught', 'bowled', 'lbw', 'stumped', 'caught and bowled', 'hit wicket']
merged_df['is_wicket'] = merged_df['dismissal_kind'].isin(wicket_kinds)
wickets = merged_df[merged_df['is_wicket']].groupby(['match_id', 'batting_team'])['is_wicket'].sum().reset_index()
wickets.rename(columns={'is_wicket': 'wickets_lost'}, inplace=True)

# Combine features
features_df = total_runs.merge(wickets, on=['match_id', 'batting_team'], how='left')
features_df['wickets_lost'] = features_df['wickets_lost'].fillna(0).astype(int)

os.makedirs('data_processing/processed_data', exist_ok=True)
features_df.to_csv('data_processing/processed_data/final_features.csv', index=False)

features_df.head()


,match_id,batting_team,innings_total_runs,wickets_lost
0,335982,Kolkata Knight Riders,222,3
1,335982,Royal Challengers Bangalore,82,9
2,335983,Chennai Super Kings,240,5
3,335983,Kings XI Punjab,207,4
4,335984,Delhi Daredevils,132,1
